In [1]:
# EXECUTE LCMV


# Imports
import os
import re
from pathlib import Path
from tqdm import tqdm
from joblib import Parallel, delayed
from lcmv_class import LCMVSourceEstimator, CONFIG_TEMPLATE, PROJECT_BASE

# === CONFIGURATION ===
ICA_BASE_PATH = Path("/home/jaizor/jaizor/xtra/derivatives/eeg/ica")
GPS_FILE_PATH = "data/ghw280_from_egig.gpsc"

# ⚙️ TOGGLE: Set to False to skip DiFuMo entirely
USE_DIFUMO = True

# DiFuMo config (only used if USE_DIFUMO=True)
DIFUMO_CONFIG = {
    'dimension': 512,
    'resolution_mm': 2
}

CONFIG_TEMPLATE.update({
    'project_base': PROJECT_BASE,
    'gpsc_file_path': GPS_FILE_PATH,
    'reg': 0.01,
    'n_jobs': 4  # per-job parallelism (for MNE internals)
})

# === HELPER FUNCTIONS ===
def extract_subject_and_task(filepath: Path):
    subject_id = None
    for part in filepath.parts:
        if part.startswith('sub-'):
            subject_id = part
            break
    if not subject_id:
        raise ValueError(f"Subject ID not found in: {filepath}")
    task = filepath.parent.name
    return subject_id, task

def process_single_file(fif_path: Path, config_template: dict, use_difumo: bool, difumo_config: dict):
    """Process one ICA file — clean single-path logic."""
    try:
        subject_id, task_name = extract_subject_and_task(fif_path)
        config = config_template.copy()
        config.update({
            'subject_id': subject_id,
            'task': task_name,
            'ica_file_path': str(fif_path.relative_to(PROJECT_BASE))
        })

        estimator = LCMVSourceEstimator(config)
        
        # 🔍 Decide what output to check for skipping
        if use_difumo:
            output_file = estimator.subject_output / "difumo_time_courses.npy"
            skip_reason = "DiFuMo"
        else:
            output_file = estimator.subject_output / "source_estimate_LCMV.h5"
            skip_reason = "LCMV"

        # ✅ Skip if final output already exists
        if output_file.exists():
            return "skipped", f"{fif_path.name} ({skip_reason} done)"

        # 🔁 Always run full LCMV pipeline (required for both modes)
        estimator.run_enhanced_computation()

        # ➕ Optionally run DiFuMo
        if use_difumo:
            estimator.run_difumo_extraction(difumo_config=difumo_config)

        status = "success (DiFuMo)" if use_difumo else "success (LCMV only)"
        return "success", f"{fif_path.name} ({status})"

    except Exception as e:
        error_type = "DiFuMo" if use_difumo else "LCMV"
        return "failed", f"{fif_path.name} — {error_type} error: {e}"

# === MAIN ===
if __name__ == "__main__":
    # Collect and sort files
    fif_files = []
    for root, _, files in os.walk(ICA_BASE_PATH):
        for file in files:
            if file.endswith('.fif'):
                fif_files.append(Path(root) / file)

    def get_subject_number(p):
        match = re.search(r'sub-(\d+)', str(p))
        return int(match.group(1)) if match else float('inf')

    fif_files.sort(key=lambda f: (get_subject_number(f), str(f)))

    mode_str = "LCMV + DiFuMo" if USE_DIFUMO else "LCMV only"
    print(f"Found {len(fif_files)} files. Running in '{mode_str}' mode...\n")

    # 🔥 PARALLEL EXECUTION
    results = Parallel(n_jobs=-1, verbose=1, backend="loky")(
        delayed(process_single_file)(
            fif_path, 
            CONFIG_TEMPLATE, 
            USE_DIFUMO, 
            DIFUMO_CONFIG
        )
        for fif_path in fif_files
    )

    # Summarize
    success = sum(1 for r, _ in results if r == "success")
    skipped = sum(1 for r, _ in results if r == "skipped")
    failed = len(results) - success - skipped

    print("\n" + "="*60)
    print(f"✅ Success:      {success}")
    print(f"⏭️  Skipped:     {skipped}")
    print(f"❌ Failed:       {failed}")
    print(f"📊 Total files:  {len(fif_files)}")
    print(f"⚙️  Mode:         {'DiFuMo + LCMV' if USE_DIFUMO else 'LCMV only'}")
    print("="*60)

    # Log failures
    if failed > 0:
        print("\n⚠️  Failed files:")
        for status, msg in results:
            if status == "failed":
                print(f"  - {msg}")

Found 35 files. Running in 'LCMV + DiFuMo' mode...



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.


🎯 ENHANCED LCMV SOURCE ESTIMATION - Subject: sub-03

=== Loading Data ===
🎯 ENHANCED LCMV SOURCE ESTIMATION - Subject: sub-08

=== Loading Data ===
🎯 ENHANCED LCMV SOURCE ESTIMATION - Subject: sub-05

=== Loading Data ===

🎯 ENHANCED LCMV SOURCE ESTIMATION - Subject: sub-01

=== Loading Data ===
🎯 ENHANCED LCMV SOURCE ESTIMATION - Subject: sub-10

=== Loading Data ===


/mnt/movement/users/jaizor/Δ/lcmv_class.py:70: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/eeg/ica/sub-03/eyes_open/rest_off_sub-03_eyes_open_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(ica_file, preload=True)
/mnt/movement/users/jaizor/Δ/lcmv_class.py:70: RuntimeWarning: This filename (/home/jaizor/jaizor/xtra/derivatives/eeg/ica/sub-05/bima_task/bima_off_sub-05_bima_task_ica.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(ica_file, preload=True)
/mnt/movement/users/jaizor/Δ/lcmv_class.py:70: RuntimeWarning: This filename (/home/jaizor/j

Data: 1.0min, 250.0Hz, 15002 samples

=== Enhanced Preprocessing Pipeline ===

=== Creating Enhanced Montage with Coordinate Normalization ===
Original mean position (mm): [100.83802817  94.83802817 166.92605634]

🔍 Checking EEG reference status...
custom_ref_applied: 1
n_projs: 0
proj_applied: False
📎 No average reference projection found. Applying it...
Data: 0.7min, 500.0Hz, 21836 samples

=== Enhanced Preprocessing Pipeline ===

=== Creating Enhanced Montage with Coordinate Normalization ===
Original mean position (mm): [100.83802817  94.83802817 166.92605634]
Data: 0.6min, 500.0Hz, 18668 samples

=== Enhanced Preprocessing Pipeline ===

=== Creating Enhanced Montage with Coordinate Normalization ===
Original mean position (mm): [100.83802817  94.83802817 166.92605634]
Data: 1.0min, 500.0Hz, 30003 samples

=== Enhanced Preprocessing Pipeline ===

=== Creating Enhanced Montage with Coordinate Normalization ===
Original mean position (mm): [100.83802817  94.83802817 166.92605634]
🎯 A

...

/mnt/movement/users/jaizor/Δ/lcmv_class.py:285: RuntimeWarning: The largest eigenvalue of the 281-channel eeg covariance (rank=280) is over 10 orders of magnitude larger than the smallest (1.29e-07 > 1e10 * 7.61e-19), the resulting whitener will likely be unstable
  filters = mne.beamformer.make_lcmv(
/mnt/movement/users/jaizor/Δ/lcmv_class.py:285: RuntimeWarning: The largest eigenvalue of the 281-channel eeg covariance (rank=280) is over 10 orders of magnitude larger than the smallest (1.29e-07 > 1e10 * 7.61e-19), the resulting whitener will likely be unstable
  filters = mne.beamformer.make_lcmv(


    EEG: rank 280 after 1 projector applied to 281 channels
Filter computation complete
Applying LCMV filters to continuous data...
Computing beamformer filters for 13222 sources
Computing rank from covariance with rank='info'
    EEG: rank 280 after 1 projector applied to 281 channels
Making LCMV beamformer with rank {'eeg': 280}
Computing inverse operator with 281 channels.
    EEG: rank 280 after 1 projector applied to 281 channels
Making LCMV beamformer with rank {'eeg': 280}
    281 out of 281 channels remain after picking
Computing beamformer filters for 13222 sources
Creating the source covariance matrix
Excluding 7 channel(s) missing from the provided forward operator and/or covariance matrices
Creating LCMV spatial filters...
Computing inverse operator with 281 channels.
    281 out of 281 channels remain after picking
Excluding 8 channel(s) missing from the provided forward operator and/or covariance matrices
Adjusting source covariance matrix.
Computing rank from covariance 

[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed: 13.3min remaining:  3.3min


✅ 505/512 components have at least one source
💾 Saved to: /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-14_eyes_open

🎉 SUCCESS: DiFuMo time series extraction complete!
📊 Output shape: (512, 30001) (512 components × 30001 time points)
📄 Details saved in:
   - /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-14_eyes_open/difumo_time_courses.npy
   - /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-14_eyes_open/difumo_component_info.csv
✅ 505/512 components have at least one source
💾 Saved to: /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-01_eyes_open

🎉 SUCCESS: DiFuMo time series extraction complete!
📊 Output shape: (512, 30003) (512 components × 30003 time points)
📄 Details saved in:
   - /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-01_eyes_open/difumo_time_courses.npy
   - /home/jaizor/jaizor/xtra/derivatives/lcmv/sub-01_eyes_open/difumo_component_info.csv
Processing component 501/512
Processing component 501/512
Processing component 501/512
✅ 505/512 components have at least one source
✅

[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed: 13.7min finished
